In [85]:
import MDAnalysis as mda
import numpy as np
import  as cluster

SyntaxError: invalid syntax (<ipython-input-85-5f4514236a02>, line 3)

In [253]:
topology_ = '../../../for_hrach/tx114/production.tpr'

trajectory_ = '../../../for_hrach/tx114/production.trr'

cluster_atom_sel = 'name C1A'

cutoff_cluster=10.0

u=mda.Universe(topology_,trajectory_)

cluster_atoms=u.select_atoms(cluster_atom_sel)
micelle=u.select_atoms('not name HW? OW and not resid 87')


In [138]:
u.trajectory[-2]
u.atoms.write('system_orginal.gro')

In [254]:
micelle_hydrophilic=micelle.select_atoms('resname TX4 and not name C1D C1C C1B C1A C1E C1F C2 C3A C3B C4 C5 C6C C6B C6A ')
micelle_hydrophobic=micelle.select_atoms('resname TX4 and name C1D C1C C1B C1A C1E C1F C2 C3A C3B C4 C5 C6C C6B C6A ')

water=u.select_atoms('name OW')
system=u.atoms

In [258]:
split_translate = np.zeros_like(system.positions)+u.dimensions[0]/2

system.translate(split_translate)

system.wrap()

array([[96.8327  , 12.727295, 10.727295],
       [95.9727  , 13.09729 , 10.137299],
       [96.522705, 11.827301, 11.297302],
       ...,
       [57.147293, 45.537308, 31.587296],
       [57.1073  , 45.127304, 30.727295],
       [58.007294, 45.277313, 31.937302]], dtype=float32)

In [266]:
system.select_atoms('not resid 87').atoms.write('micelle_test.gro')


NotImplementedError: MOL2Writer cannot currently write non MOL2 data

In [264]:

u=mda.Universe(topology_,'micelle_test.gro')


/Users/rmz/opt/anaconda3/envs/umap/lib/python3.7/site-packages/MDAnalysis/topology/tpr/utils.py:395: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  category=DeprecationWarning)


ValueError: The topology and GRO trajectory files don't have the same number of atoms!
Topology number of atoms 169824
Trajectory: micelle_test.gro Number of atoms 169731

# micelle unwrapping

In [262]:
#save original system bonds
bonds_real = system.atoms.bonds
print(bonds_real)

<TopologyGroup containing 117866 bonds>


In [240]:
bonds_new = []

micelle_resids = np.unique(micelle.resids)
micelle_indices = [u.select_atoms('resid '+str(res))[0].index for res in micelle_resids]

for ind in micelle_indices:
    for ind_prime in micelle_indices:
        if ind_prime>ind:
            bonds_new.append((ind,ind_prime))

In [241]:
water_indices = u.select_atoms('name OW').indices


for ind_prime in water_indices:

    bonds_new.append((micelle_indices[0],ind_prime))

In [242]:
u.add_bonds(bonds_new)

In [243]:
u.atoms.bonds

<TopologyGroup containing 180999 bonds>

In [244]:
u.atoms.unwrap()

u.atoms.translate(-np.array([min(u.atoms.positions[:,0]),min(u.atoms.positions[:,1]),min(u.atoms.positions[:,2])]))






<AtomGroup with 169824 atoms>

In [249]:
dims = ['x','y','z']

for d in range(1):
    vec = np.array([0.,0.,0.])
    vec[d] += u.dimensions[d]
    ind1 = u.select_atoms('name OW and prop '+str(dims[d])+' < '+str(micelle_com[d]-u.dimensions[d]/2)).indices
    u.atoms[ind1].translate(vec)
    ind2 = u.select_atoms('name OW and prop '+str(dims[d])+' > '+str(micelle_com[d]-u.dimensions[d]/2)).indices
    u.atoms[ind2].translate(-vec)

In [250]:
u.atoms.write('system_test2.gro')


In [252]:
micelle_com[d]-u.dimensions[d]/2

1.7449447994034202

In [172]:
u.select_atoms('name OW and prop x < '+str(micelle_com[0]-u.dimensions[0]/2)).atoms.positions[:,0]+=u.dimensions[0]

In [181]:

micelle_com,u.dimensions[0]/2,str(micelle_com[0]-u.dimensions[0]/2)

(array([61.23559879, 57.6315517 , 57.89425468]),
 59.49065399169922,
 '1.7449447994034202')

In [154]:
micelle_com = micelle.center_of_mass()
water=u.select_atoms('name OW')


for mol in range(0,len(water.positions),3):
    for d in range(3):
        if water.positions[mol][d] - micelle_com[d]>u.dimensions[d]/2:
#             print(d,water.resids[mol],water.positions[mol][d],micelle_com[d])
            
            water.positions[mol][d]+=water.positions[mol][d]-u.dimensions[d]
        
        elif water.positions[mol][d] - micelle_com[d]<u.dimensions[d]/2:
#             print(d,water.resids[mol],water.positions[mol][d],micelle_com[d])
            
            water.positions[mol][d]+=water.positions[mol][d]+u.dimensions[d]
    print(mol)

0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297
300
303
306
309
312
315
318
321
324
327
330
333
336
339
342
345
348
351
354
357
360
363
366
369
372
375
378
381
384
387
390
393
396
399
402
405
408
411
414
417
420
423
426
429
432
435
438
441
444
447
450
453
456
459
462
465
468
471
474
477
480
483
486
489
492
495
498
501
504
507
510
513
516
519
522
525
528
531
534
537
540
543
546
549
552
555
558
561
564
567
570
573
576
579
582
585
588
591
594
597
600
603
606
609
612
615
618
621
624
627
630
633
636
639
642
645
648
651
654
657
660
663
666
669
672
675
678
681
684
687
690
693
696
699
702
705
708
711
714
717
720
723
726
729
732
735
738
741
744
747
750
753
756
759
762
765
768
771
774
77

5142
5145
5148
5151
5154
5157
5160
5163
5166
5169
5172
5175
5178
5181
5184
5187
5190
5193
5196
5199
5202
5205
5208
5211
5214
5217
5220
5223
5226
5229
5232
5235
5238
5241
5244
5247
5250
5253
5256
5259
5262
5265
5268
5271
5274
5277
5280
5283
5286
5289
5292
5295
5298
5301
5304
5307
5310
5313
5316
5319
5322
5325
5328
5331
5334
5337
5340
5343
5346
5349
5352
5355
5358
5361
5364
5367
5370
5373
5376
5379
5382
5385
5388
5391
5394
5397
5400
5403
5406
5409
5412
5415
5418
5421
5424
5427
5430
5433
5436
5439
5442
5445
5448
5451
5454
5457
5460
5463
5466
5469
5472
5475
5478
5481
5484
5487
5490
5493
5496
5499
5502
5505
5508
5511
5514
5517
5520
5523
5526
5529
5532
5535
5538
5541
5544
5547
5550
5553
5556
5559
5562
5565
5568
5571
5574
5577
5580
5583
5586
5589
5592
5595
5598
5601
5604
5607
5610
5613
5616
5619
5622
5625
5628
5631
5634
5637
5640
5643
5646
5649
5652
5655
5658
5661
5664
5667
5670
5673
5676
5679
5682
5685
5688
5691
5694
5697
5700
5703
5706
5709
5712
5715
5718
5721
5724
5727
5730
5733
5736
5739


10128
10131
10134
10137
10140
10143
10146
10149
10152
10155
10158
10161
10164
10167
10170
10173
10176
10179
10182
10185
10188
10191
10194
10197
10200
10203
10206
10209
10212
10215
10218
10221
10224
10227
10230
10233
10236
10239
10242
10245
10248
10251
10254
10257
10260
10263
10266
10269
10272
10275
10278
10281
10284
10287
10290
10293
10296
10299
10302
10305
10308
10311
10314
10317
10320
10323
10326
10329
10332
10335
10338
10341
10344
10347
10350
10353
10356
10359
10362
10365
10368
10371
10374
10377
10380
10383
10386
10389
10392
10395
10398
10401
10404
10407
10410
10413
10416
10419
10422
10425
10428
10431
10434
10437
10440
10443
10446
10449
10452
10455
10458
10461
10464
10467
10470
10473
10476
10479
10482
10485
10488
10491
10494
10497
10500
10503
10506
10509
10512
10515
10518
10521
10524
10527
10530
10533
10536
10539
10542
10545
10548
10551
10554
10557
10560
10563
10566
10569
10572
10575
10578
10581
10584
10587
10590
10593
10596
10599
10602
10605
10608
10611
10614
10617
10620
10623
1062

14292
14295
14298
14301
14304
14307
14310
14313
14316
14319
14322
14325
14328
14331
14334
14337
14340
14343
14346
14349
14352
14355
14358
14361
14364
14367
14370
14373
14376
14379
14382
14385
14388
14391
14394
14397
14400
14403
14406
14409
14412
14415
14418
14421
14424
14427
14430
14433
14436
14439
14442
14445
14448
14451
14454
14457
14460
14463
14466
14469
14472
14475
14478
14481
14484
14487
14490
14493
14496
14499
14502
14505
14508
14511
14514
14517
14520
14523
14526
14529
14532
14535
14538
14541
14544
14547
14550
14553
14556
14559
14562
14565
14568
14571
14574
14577
14580
14583
14586
14589
14592
14595
14598
14601
14604
14607
14610
14613
14616
14619
14622
14625
14628
14631
14634
14637
14640
14643
14646
14649
14652
14655
14658
14661
14664
14667
14670
14673
14676
14679
14682
14685
14688
14691
14694
14697
14700
14703
14706
14709
14712
14715
14718
14721
14724
14727
14730
14733
14736
14739
14742
14745
14748
14751
14754
14757
14760
14763
14766
14769
14772
14775
14778
14781
14784
14787
1479

18444
18447
18450
18453
18456
18459
18462
18465
18468
18471
18474
18477
18480
18483
18486
18489
18492
18495
18498
18501
18504
18507
18510
18513
18516
18519
18522
18525
18528
18531
18534
18537
18540
18543
18546
18549
18552
18555
18558
18561
18564
18567
18570
18573
18576
18579
18582
18585
18588
18591
18594
18597
18600
18603
18606
18609
18612
18615
18618
18621
18624
18627
18630
18633
18636
18639
18642
18645
18648
18651
18654
18657
18660
18663
18666
18669
18672
18675
18678
18681
18684
18687
18690
18693
18696
18699
18702
18705
18708
18711
18714
18717
18720
18723
18726
18729
18732
18735
18738
18741
18744
18747
18750
18753
18756
18759
18762
18765
18768
18771
18774
18777
18780
18783
18786
18789
18792
18795
18798
18801
18804
18807
18810
18813
18816
18819
18822
18825
18828
18831
18834
18837
18840
18843
18846
18849
18852
18855
18858
18861
18864
18867
18870
18873
18876
18879
18882
18885
18888
18891
18894
18897
18900
18903
18906
18909
18912
18915
18918
18921
18924
18927
18930
18933
18936
18939
1894

...

30804
30807
30810
30813
30816
30819
30822
30825
30828
30831
30834
30837
30840
30843
30846
30849
30852
30855
30858
30861
30864
30867
30870
30873
30876
30879
30882
30885
30888
30891
30894
30897
30900
30903
30906
30909
30912
30915
30918
30921
30924
30927
30930
30933
30936
30939
30942
30945
30948
30951
30954
30957
30960
30963
30966
30969
30972
30975
30978
30981
30984
30987
30990
30993
30996
30999
31002
31005
31008
31011
31014
31017
31020
31023
31026
31029
31032
31035
31038
31041
31044
31047
31050
31053
31056
31059
31062
31065
31068
31071
31074
31077
31080
31083
31086
31089
31092
31095
31098
31101
31104
31107
31110
31113
31116
31119
31122
31125
31128
31131
31134
31137
31140
31143
31146
31149
31152
31155
31158
31161
31164
31167
31170
31173
31176
31179
31182
31185
31188
31191
31194
31197
31200
31203
31206
31209
31212
31215
31218
31221
31224
31227
31230
31233
31236
31239
31242
31245
31248
31251
31254
31257
31260
31263
31266
31269
31272
31275
31278
31281
31284
31287
31290
31293
31296
31299
3130

34995
34998
35001
35004
35007
35010
35013
35016
35019
35022
35025
35028
35031
35034
35037
35040
35043
35046
35049
35052
35055
35058
35061
35064
35067
35070
35073
35076
35079
35082
35085
35088
35091
35094
35097
35100
35103
35106
35109
35112
35115
35118
35121
35124
35127
35130
35133
35136
35139
35142
35145
35148
35151
35154
35157
35160
35163
35166
35169
35172
35175
35178
35181
35184
35187
35190
35193
35196
35199
35202
35205
35208
35211
35214
35217
35220
35223
35226
35229
35232
35235
35238
35241
35244
35247
35250
35253
35256
35259
35262
35265
35268
35271
35274
35277
35280
35283
35286
35289
35292
35295
35298
35301
35304
35307
35310
35313
35316
35319
35322
35325
35328
35331
35334
35337
35340
35343
35346
35349
35352
35355
35358
35361
35364
35367
35370
35373
35376
35379
35382
35385
35388
35391
35394
35397
35400
35403
35406
35409
35412
35415
35418
35421
35424
35427
35430
35433
35436
35439
35442
35445
35448
35451
35454
35457
35460
35463
35466
35469
35472
35475
35478
35481
35484
35487
35490
3549

39189
39192
39195
39198
39201
39204
39207
39210
39213
39216
39219
39222
39225
39228
39231
39234
39237
39240
39243
39246
39249
39252
39255
39258
39261
39264
39267
39270
39273
39276
39279
39282
39285
39288
39291
39294
39297
39300
39303
39306
39309
39312
39315
39318
39321
39324
39327
39330
39333
39336
39339
39342
39345
39348
39351
39354
39357
39360
39363
39366
39369
39372
39375
39378
39381
39384
39387
39390
39393
39396
39399
39402
39405
39408
39411
39414
39417
39420
39423
39426
39429
39432
39435
39438
39441
39444
39447
39450
39453
39456
39459
39462
39465
39468
39471
39474
39477
39480
39483
39486
39489
39492
39495
39498
39501
39504
39507
39510
39513
39516
39519
39522
39525
39528
39531
39534
39537
39540
39543
39546
39549
39552
39555
39558
39561
39564
39567
39570
39573
39576
39579
39582
39585
39588
39591
39594
39597
39600
39603
39606
39609
39612
39615
39618
39621
39624
39627
39630
39633
39636
39639
39642
39645
39648
39651
39654
39657
39660
39663
39666
39669
39672
39675
39678
39681
39684
3968

43356
43359
43362
43365
43368
43371
43374
43377
43380
43383
43386
43389
43392
43395
43398
43401
43404
43407
43410
43413
43416
43419
43422
43425
43428
43431
43434
43437
43440
43443
43446
43449
43452
43455
43458
43461
43464
43467
43470
43473
43476
43479
43482
43485
43488
43491
43494
43497
43500
43503
43506
43509
43512
43515
43518
43521
43524
43527
43530
43533
43536
43539
43542
43545
43548
43551
43554
43557
43560
43563
43566
43569
43572
43575
43578
43581
43584
43587
43590
43593
43596
43599
43602
43605
43608
43611
43614
43617
43620
43623
43626
43629
43632
43635
43638
43641
43644
43647
43650
43653
43656
43659
43662
43665
43668
43671
43674
43677
43680
43683
43686
43689
43692
43695
43698
43701
43704
43707
43710
43713
43716
43719
43722
43725
43728
43731
43734
43737
43740
43743
43746
43749
43752
43755
43758
43761
43764
43767
43770
43773
43776
43779
43782
43785
43788
43791
43794
43797
43800
43803
43806
43809
43812
43815
43818
43821
43824
43827
43830
43833
43836
43839
43842
43845
43848
43851
4385

KeyboardInterrupt: 

In [128]:
np.unique(system.resids)

array([    0,     1,     2, ..., 52105, 52106, 52107])

In [97]:
system.write('system_test.gro')


In [108]:
system.positions[150:160]

(array([[87.28131 , 35.499985, 74.63131 ],
        [88.14131 , 36.19998 , 74.56131 ],
        [87.03131 , 35.38999 , 75.71131 ],
        [87.86131 , 34.20999 , 74.08131 ],
        [87.941315, 34.339996, 72.98131 ],
        [88.911316, 33.92    , 74.27131 ],
        [86.98131 , 33.12999 , 74.41131 ],
        [87.35132 , 31.97    , 73.7113  ],
        [87.35132 , 32.069996, 72.61131 ],
        [88.411316, 31.769985, 73.98131 ]], dtype=float32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

59.49065399169922